## User parser agent Tests

In [ ]:
from agents.user_parser_agent import invoke_user_parser_agent

print("Running REMAS: user_parser_agent")
user_input = input("Type your message: ")

invoke_user_parser_agent(user_input=user_input)

Running REMAS: run_user_parser_agent
✅ Upserted 1 points into 'user_agent_listings' without resetting the collection.
✅ Uploaded user query with ID: 6e9fd5b8e158c408a80980e25e049dac to 'user_agent_listings'


'6e9fd5b8e158c408a80980e25e049dac'

In [3]:
from utils.qdrant_connection import count_qdrant
print("user Qdrant count:", count_qdrant("user_agent_listings"))

user Qdrant count: 1001


In [1]:
from utils.qdrant_connection import get_point_by_id
point_id = input("Enter the point ID to retrieve: ")
point = get_point_by_id("user_agent_listings", point_id)
if point:
    print(f"\n📌 Point ID: {point.id}")
    print("Payload:", point.payload)
else:
    print(f"No point found with ID: {point_id}")    
    


📌 Point ID: 6e9fd5b8-e158-c408-a809-80e25e049dac
Payload: {'listing_id': '6e9fd5b8e158c408a80980e25e049dac', 'state': ['Brooklyn', 'Queens'], 'picture_url': None, 'price': 2200, 'bedrooms': 2, 'available_from': 'October', 'soft_attributes': 'pet-friendly, close to public transportation, quiet neighborhood', 'source': 'user_query'}


## Owner parser agent Tests

In [1]:
from agents.owner_parser_agent import invoke_owner_parser_agent

print("Running REMAS: owner_parser_agent")
owner_input = input("Type your message: ")

invoke_owner_parser_agent(owner_input=owner_input)


Running REMAS: owner_parser_agent
✅ Upserted 1 points into 'owner_agent_listings' without resetting the collection.
✅ Uploaded owner listing with ID: 140d392e74131263a9ca770865306117 to 'owner_agent_listings'


'140d392e74131263a9ca770865306117'

In [4]:
from utils.qdrant_connection import count_qdrant
print("owner Qdrant count:", count_qdrant("owner_agent_listings"))

owner Qdrant count: 1008


In [ ]:
from utils.qdrant_connection import get_point_by_id
point_id = input("Enter the point ID to retrieve: ")
point = get_point_by_id("owner_agent_listings", point_id)
if point:
    print(f"\n📌 Point ID: {point.id}")
    print("Payload:", point.payload)
else:
    print(f"No point found with ID: {point_id}")    